In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from textblob import TextBlob
from nltk import tokenize

import math

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('clean_vader_textblob.csv')
df.drop(columns='Unnamed: 0',axis=1,inplace=True)
df

,Unnamed: 0.1,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,source,is_retweet,random_sample,lang,cleaned_text,vader,compound,sentiment_vader,polarity,sentiment_textblob
0,6,Bitcoin-Bot,"Florida, USA",Bot to generate Bitcoin picture as combination...,2019-12-23 16:49:16,131.0,84.0,5728.0,False,2021-02-10 23:53:17,...,BTC_p_bot,False,Yes,en,lt fire amp man gt,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL
1,8,Mikcoin,NaN,"Technical Analyst | Trader\n\nNo certainty, on...",2020-11-26 23:45:46,104.0,41.0,238.0,False,2021-02-10 23:52:25,...,Twitter Web App,False,Yes,en,rsr i know i told you guys the target was i k...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL
2,23,Alex Jiménez,"Las Vegas, NV",Chief Strategy Officer @Extractable #Fintech #...,2012-04-25 15:26:53,14469.0,822.0,40097.0,True,2021-02-10 23:45:03,...,Hootsuite Inc.,False,Yes,en,s rally signals the rise of digital assets th...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",0.1779,POSITIVE,-0.200000,NEGATIVE
3,44,FinTech News Hub,United States,FinTech News Hub is a Blockchain and Cryptocur...,2016-09-29 12:32:04,348.0,126.0,638.0,False,2021-02-10 23:33:55,...,Paper.li,False,Yes,en,our featured article for today please don t fo...,"{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'comp...",0.5037,POSITIVE,0.000000,NEUTRAL
4,47,Tonya Barris,NaN,NaN,2020-12-29 16:55:33,0.0,16.0,2.0,False,2021-02-10 23:32:35,...,Twitter for iPhone,False,Yes,en,who want to learn about forex trading and join...,"{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'comp...",0.7650,POSITIVE,0.800000,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51854,2569336,Emilio3r ⚡LOCK MEX MONSTA⚡,trois-rivières,"Enthousiaste de la crypto, de la bière de micr...",2010-09-30 18:10:30,149.0,347.0,2685.0,False,2022-03-14 19:40:26,...,Twitter for Android,False,Yes,en,recession the economical shift is coming the ...,"{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'comp...",-0.7707,NEGATIVE,0.237143,POSITIVE
51855,2569345,al_riba,NaN,"Matematik Öğretmeni, Teknoloji meraklısı, Epey...",2014-02-28 19:15:54,202.0,205.0,8332.0,False,2022-03-14 19:39:50,...,Twitter for Android,False,Yes,en,we re going to the moon don t be late,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,-0.300000,NEGATIVE
51856,2569357,SOUNDSM,NaN,близнец,2021-09-10 17:47:34,8.0,81.0,80.0,False,2022-03-14 19:38:53,...,Twitter Web App,False,Yes,en,let s stop the war together proceeds from the ...,"{'neg': 0.282, 'neu': 0.718, 'pos': 0.0, 'comp...",-0.9117,NEGATIVE,0.000000,NEUTRAL
51857,2569371,HODL never SODL 丰 🇸🇻 🍌🍞,El Salvador,HODLer of last resort & unlicensed node operat...,2020-08-12 10:31:54,4315.0,2448.0,107327.0,False,2022-03-14 19:37:46,...,Twitter Web App,False,Yes,en,the majority of users will be just watching t...,"{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compou...",0.3612,POSITIVE,0.200000,POSITIVE


# Impact Score

In [4]:
def generate_impact_score(compound_tweet) : 
    coef_verified = 2 if compound_tweet.user_verified else 1
    log_follower = math.log10(compound_tweet.user_followers +1)
    impact_score = coef_verified * log_follower
    return impact_score

df['impact_score_ver_foll'] = df.apply(generate_impact_score,axis=1)

In [5]:
df

,Unnamed: 0.1,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,is_retweet,random_sample,lang,cleaned_text,vader,compound,sentiment_vader,polarity,sentiment_textblob,impact_score_ver_foll
0,6,Bitcoin-Bot,"Florida, USA",Bot to generate Bitcoin picture as combination...,2019-12-23 16:49:16,131.0,84.0,5728.0,False,2021-02-10 23:53:17,...,False,Yes,en,lt fire amp man gt,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.120574
1,8,Mikcoin,NaN,"Technical Analyst | Trader\n\nNo certainty, on...",2020-11-26 23:45:46,104.0,41.0,238.0,False,2021-02-10 23:52:25,...,False,Yes,en,rsr i know i told you guys the target was i k...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.021189
2,23,Alex Jiménez,"Las Vegas, NV",Chief Strategy Officer @Extractable #Fintech #...,2012-04-25 15:26:53,14469.0,822.0,40097.0,True,2021-02-10 23:45:03,...,False,Yes,en,s rally signals the rise of digital assets th...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",0.1779,POSITIVE,-0.200000,NEGATIVE,8.320937
3,44,FinTech News Hub,United States,FinTech News Hub is a Blockchain and Cryptocur...,2016-09-29 12:32:04,348.0,126.0,638.0,False,2021-02-10 23:33:55,...,False,Yes,en,our featured article for today please don t fo...,"{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'comp...",0.5037,POSITIVE,0.000000,NEUTRAL,2.542825
4,47,Tonya Barris,NaN,NaN,2020-12-29 16:55:33,0.0,16.0,2.0,False,2021-02-10 23:32:35,...,False,Yes,en,who want to learn about forex trading and join...,"{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'comp...",0.7650,POSITIVE,0.800000,POSITIVE,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51854,2569336,Emilio3r ⚡LOCK MEX MONSTA⚡,trois-rivières,"Enthousiaste de la crypto, de la bière de micr...",2010-09-30 18:10:30,149.0,347.0,2685.0,False,2022-03-14 19:40:26,...,False,Yes,en,recession the economical shift is coming the ...,"{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'comp...",-0.7707,NEGATIVE,0.237143,POSITIVE,2.176091
51855,2569345,al_riba,NaN,"Matematik Öğretmeni, Teknoloji meraklısı, Epey...",2014-02-28 19:15:54,202.0,205.0,8332.0,False,2022-03-14 19:39:50,...,False,Yes,en,we re going to the moon don t be late,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,-0.300000,NEGATIVE,2.307496
51856,2569357,SOUNDSM,NaN,близнец,2021-09-10 17:47:34,8.0,81.0,80.0,False,2022-03-14 19:38:53,...,False,Yes,en,let s stop the war together proceeds from the ...,"{'neg': 0.282, 'neu': 0.718, 'pos': 0.0, 'comp...",-0.9117,NEGATIVE,0.000000,NEUTRAL,0.954243
51857,2569371,HODL never SODL 丰 🇸🇻 🍌🍞,El Salvador,HODLer of last resort & unlicensed node operat...,2020-08-12 10:31:54,4315.0,2448.0,107327.0,False,2022-03-14 19:37:46,...,False,Yes,en,the majority of users will be just watching t...,"{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compou...",0.3612,POSITIVE,0.200000,POSITIVE,3.635081


# Impact Score With Vader

In [6]:
def generate_impact_var(compound_tweet) : 
    coef_verified = 2 if compound_tweet.user_verified else 1
    log_follower = math.log10(compound_tweet.user_followers +1)
    impact_score = coef_verified * log_follower * compound_tweet.compound
    return impact_score

df['impact_score_var'] = df.apply(generate_impact_var,axis=1)

In [7]:
df

,Unnamed: 0.1,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,random_sample,lang,cleaned_text,vader,compound,sentiment_vader,polarity,sentiment_textblob,impact_score_ver_foll,impact_score_var
0,6,Bitcoin-Bot,"Florida, USA",Bot to generate Bitcoin picture as combination...,2019-12-23 16:49:16,131.0,84.0,5728.0,False,2021-02-10 23:53:17,...,Yes,en,lt fire amp man gt,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.120574,0.000000
1,8,Mikcoin,NaN,"Technical Analyst | Trader\n\nNo certainty, on...",2020-11-26 23:45:46,104.0,41.0,238.0,False,2021-02-10 23:52:25,...,Yes,en,rsr i know i told you guys the target was i k...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.021189,0.000000
2,23,Alex Jiménez,"Las Vegas, NV",Chief Strategy Officer @Extractable #Fintech #...,2012-04-25 15:26:53,14469.0,822.0,40097.0,True,2021-02-10 23:45:03,...,Yes,en,s rally signals the rise of digital assets th...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",0.1779,POSITIVE,-0.200000,NEGATIVE,8.320937,1.480295
3,44,FinTech News Hub,United States,FinTech News Hub is a Blockchain and Cryptocur...,2016-09-29 12:32:04,348.0,126.0,638.0,False,2021-02-10 23:33:55,...,Yes,en,our featured article for today please don t fo...,"{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'comp...",0.5037,POSITIVE,0.000000,NEUTRAL,2.542825,1.280821
4,47,Tonya Barris,NaN,NaN,2020-12-29 16:55:33,0.0,16.0,2.0,False,2021-02-10 23:32:35,...,Yes,en,who want to learn about forex trading and join...,"{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'comp...",0.7650,POSITIVE,0.800000,POSITIVE,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51854,2569336,Emilio3r ⚡LOCK MEX MONSTA⚡,trois-rivières,"Enthousiaste de la crypto, de la bière de micr...",2010-09-30 18:10:30,149.0,347.0,2685.0,False,2022-03-14 19:40:26,...,Yes,en,recession the economical shift is coming the ...,"{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'comp...",-0.7707,NEGATIVE,0.237143,POSITIVE,2.176091,-1.677114
51855,2569345,al_riba,NaN,"Matematik Öğretmeni, Teknoloji meraklısı, Epey...",2014-02-28 19:15:54,202.0,205.0,8332.0,False,2022-03-14 19:39:50,...,Yes,en,we re going to the moon don t be late,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,-0.300000,NEGATIVE,2.307496,0.000000
51856,2569357,SOUNDSM,NaN,близнец,2021-09-10 17:47:34,8.0,81.0,80.0,False,2022-03-14 19:38:53,...,Yes,en,let s stop the war together proceeds from the ...,"{'neg': 0.282, 'neu': 0.718, 'pos': 0.0, 'comp...",-0.9117,NEGATIVE,0.000000,NEUTRAL,0.954243,-0.869983
51857,2569371,HODL never SODL 丰 🇸🇻 🍌🍞,El Salvador,HODLer of last resort & unlicensed node operat...,2020-08-12 10:31:54,4315.0,2448.0,107327.0,False,2022-03-14 19:37:46,...,Yes,en,the majority of users will be just watching t...,"{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compou...",0.3612,POSITIVE,0.200000,POSITIVE,3.635081,1.312991


# Impact Score with Textblob

In [8]:
def generate_impact_tb(compound_tweet) : 
    coef_verified = 2 if compound_tweet.user_verified else 1
    log_follower = math.log10(compound_tweet.user_followers +1)
    impact_score = coef_verified * log_follower * compound_tweet.polarity
    return impact_score

df['impact_score_tb'] = df.apply(generate_impact_tb,axis=1)

In [9]:
df

,Unnamed: 0.1,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,lang,cleaned_text,vader,compound,sentiment_vader,polarity,sentiment_textblob,impact_score_ver_foll,impact_score_var,impact_score_tb
0,6,Bitcoin-Bot,"Florida, USA",Bot to generate Bitcoin picture as combination...,2019-12-23 16:49:16,131.0,84.0,5728.0,False,2021-02-10 23:53:17,...,en,lt fire amp man gt,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.120574,0.000000,0.000000
1,8,Mikcoin,NaN,"Technical Analyst | Trader\n\nNo certainty, on...",2020-11-26 23:45:46,104.0,41.0,238.0,False,2021-02-10 23:52:25,...,en,rsr i know i told you guys the target was i k...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,0.000000,NEUTRAL,2.021189,0.000000,0.000000
2,23,Alex Jiménez,"Las Vegas, NV",Chief Strategy Officer @Extractable #Fintech #...,2012-04-25 15:26:53,14469.0,822.0,40097.0,True,2021-02-10 23:45:03,...,en,s rally signals the rise of digital assets th...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",0.1779,POSITIVE,-0.200000,NEGATIVE,8.320937,1.480295,-1.664187
3,44,FinTech News Hub,United States,FinTech News Hub is a Blockchain and Cryptocur...,2016-09-29 12:32:04,348.0,126.0,638.0,False,2021-02-10 23:33:55,...,en,our featured article for today please don t fo...,"{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'comp...",0.5037,POSITIVE,0.000000,NEUTRAL,2.542825,1.280821,0.000000
4,47,Tonya Barris,NaN,NaN,2020-12-29 16:55:33,0.0,16.0,2.0,False,2021-02-10 23:32:35,...,en,who want to learn about forex trading and join...,"{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'comp...",0.7650,POSITIVE,0.800000,POSITIVE,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51854,2569336,Emilio3r ⚡LOCK MEX MONSTA⚡,trois-rivières,"Enthousiaste de la crypto, de la bière de micr...",2010-09-30 18:10:30,149.0,347.0,2685.0,False,2022-03-14 19:40:26,...,en,recession the economical shift is coming the ...,"{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'comp...",-0.7707,NEGATIVE,0.237143,POSITIVE,2.176091,-1.677114,0.516044
51855,2569345,al_riba,NaN,"Matematik Öğretmeni, Teknoloji meraklısı, Epey...",2014-02-28 19:15:54,202.0,205.0,8332.0,False,2022-03-14 19:39:50,...,en,we re going to the moon don t be late,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL,-0.300000,NEGATIVE,2.307496,0.000000,-0.692249
51856,2569357,SOUNDSM,NaN,близнец,2021-09-10 17:47:34,8.0,81.0,80.0,False,2022-03-14 19:38:53,...,en,let s stop the war together proceeds from the ...,"{'neg': 0.282, 'neu': 0.718, 'pos': 0.0, 'comp...",-0.9117,NEGATIVE,0.000000,NEUTRAL,0.954243,-0.869983,0.000000
51857,2569371,HODL never SODL 丰 🇸🇻 🍌🍞,El Salvador,HODLer of last resort & unlicensed node operat...,2020-08-12 10:31:54,4315.0,2448.0,107327.0,False,2022-03-14 19:37:46,...,en,the majority of users will be just watching t...,"{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compou...",0.3612,POSITIVE,0.200000,POSITIVE,3.635081,1.312991,0.727016


# All data

In [12]:
data = df[["date","text","cleaned_text","user_followers", "user_verified", "compound","polarity","impact_score_ver_foll","impact_score_var","impact_score_tb"]]

In [13]:
data

,date,text,cleaned_text,user_followers,user_verified,compound,polarity,impact_score_ver_foll,impact_score_var,impact_score_tb
0,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...,lt fire amp man gt,131.0,False,0.0000,0.000000,2.120574,0.000000,0.000000
1,2021-02-10 23:52:25,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,rsr i know i told you guys the target was i k...,104.0,False,0.0000,0.000000,2.021189,0.000000,0.000000
2,2021-02-10 23:45:03,#Bitcoin’s Rally Signals the Rise of Digital A...,s rally signals the rise of digital assets th...,14469.0,True,0.1779,-0.200000,8.320937,1.480295,-1.664187
3,2021-02-10 23:33:55,Our featured article for today! Please don't f...,our featured article for today please don t fo...,348.0,False,0.5037,0.000000,2.542825,1.280821,0.000000
4,2021-02-10 23:32:35,Who want to learn about Forex trading and join...,who want to learn about forex trading and join...,0.0,False,0.7650,0.800000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
51854,2022-03-14 19:40:26,"2- recession: the economical shift is coming, ...",recession the economical shift is coming the ...,149.0,False,-0.7707,0.237143,2.176091,-1.677114,0.516044
51855,2022-03-14 19:39:50,"@LadyofCrypto1 We're going to the moon, don't ...",we re going to the moon don t be late,202.0,False,0.0000,-0.300000,2.307496,0.000000,-0.692249
51856,2022-03-14 19:38:53,🛑Let's stop the war together!🛑\n📡Proceeds from...,let s stop the war together proceeds from the ...,8.0,False,-0.9117,0.000000,0.954243,-0.869983,0.000000
51857,2022-03-14 19:37:46,@patrickluberus @WhyyBitcoin @sathoarder @LOST...,the majority of users will be just watching t...,4315.0,False,0.3612,0.200000,3.635081,1.312991,0.727016


In [11]:
# df.to_csv('impact_score.csv')

In [18]:
data['date'] = pd.to_datetime(data['date'])

# AVG IMPACT SCORE

In [29]:
data['impact_score_avg'] = data.groupby(data['date'].dt.date)['impact_score_ver_foll'].transform('mean')
data_impact_only = data[['date',
                            'text',
                            'cleaned_text',
                            'user_followers',
                            'user_verified',
                            'impact_score_ver_foll',
                            'impact_score_avg'
                            ]]
data_impact_only

,date,text,cleaned_text,user_followers,user_verified,impact_score_ver_foll,impact_score_avg
0,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...,lt fire amp man gt,131.0,False,2.120574,2.581671
1,2021-02-10 23:52:25,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,rsr i know i told you guys the target was i k...,104.0,False,2.021189,2.581671
2,2021-02-10 23:45:03,#Bitcoin’s Rally Signals the Rise of Digital A...,s rally signals the rise of digital assets th...,14469.0,True,8.320937,2.581671
3,2021-02-10 23:33:55,Our featured article for today! Please don't f...,our featured article for today please don t fo...,348.0,False,2.542825,2.581671
4,2021-02-10 23:32:35,Who want to learn about Forex trading and join...,who want to learn about forex trading and join...,0.0,False,0.000000,2.581671
...,...,...,...,...,...,...,...
51854,2022-03-14 19:40:26,"2- recession: the economical shift is coming, ...",recession the economical shift is coming the ...,149.0,False,2.176091,2.557026
51855,2022-03-14 19:39:50,"@LadyofCrypto1 We're going to the moon, don't ...",we re going to the moon don t be late,202.0,False,2.307496,2.557026
51856,2022-03-14 19:38:53,🛑Let's stop the war together!🛑\n📡Proceeds from...,let s stop the war together proceeds from the ...,8.0,False,0.954243,2.557026
51857,2022-03-14 19:37:46,@patrickluberus @WhyyBitcoin @sathoarder @LOST...,the majority of users will be just watching t...,4315.0,False,3.635081,2.557026


# AVG IMPACT SCORE WITH VADER

In [34]:
data['impact_score_var_avg'] = data.groupby(data['date'].dt.date)['impact_score_var'].transform('mean')
data_impact_var = data[['date',
                            'text',
                            'cleaned_text',
                            'user_followers',
                            'user_verified',
                            'compound',
                            'impact_score_var',
                            'impact_score_var_avg'
                            ]]
data_impact_var

,date,text,cleaned_text,user_followers,user_verified,compound,impact_score_var,impact_score_var_avg
0,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...,lt fire amp man gt,131.0,False,0.0000,0.000000,0.304796
1,2021-02-10 23:52:25,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,rsr i know i told you guys the target was i k...,104.0,False,0.0000,0.000000,0.304796
2,2021-02-10 23:45:03,#Bitcoin’s Rally Signals the Rise of Digital A...,s rally signals the rise of digital assets th...,14469.0,True,0.1779,1.480295,0.304796
3,2021-02-10 23:33:55,Our featured article for today! Please don't f...,our featured article for today please don t fo...,348.0,False,0.5037,1.280821,0.304796
4,2021-02-10 23:32:35,Who want to learn about Forex trading and join...,who want to learn about forex trading and join...,0.0,False,0.7650,0.000000,0.304796
...,...,...,...,...,...,...,...,...
51854,2022-03-14 19:40:26,"2- recession: the economical shift is coming, ...",recession the economical shift is coming the ...,149.0,False,-0.7707,-1.677114,0.529045
51855,2022-03-14 19:39:50,"@LadyofCrypto1 We're going to the moon, don't ...",we re going to the moon don t be late,202.0,False,0.0000,0.000000,0.529045
51856,2022-03-14 19:38:53,🛑Let's stop the war together!🛑\n📡Proceeds from...,let s stop the war together proceeds from the ...,8.0,False,-0.9117,-0.869983,0.529045
51857,2022-03-14 19:37:46,@patrickluberus @WhyyBitcoin @sathoarder @LOST...,the majority of users will be just watching t...,4315.0,False,0.3612,1.312991,0.529045


# AVG IMPACT SCORE WITH TEXTBLOB

In [32]:
data['impact_score_tb_avg'] = data.groupby(data['date'].dt.date)['impact_score_tb'].transform('mean')
data_impact_tb = data[['date',
                            'text',
                            'cleaned_text',
                            'user_followers',
                            'user_verified',
                            'polarity',
                            'impact_score_tb',
                            'impact_score_tb_avg'
                            ]]
data_impact_tb

,date,text,cleaned_text,user_followers,user_verified,polarity,impact_score_tb,impact_score_tb_avg
0,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...,lt fire amp man gt,131.0,False,0.000000,0.000000,0.193725
1,2021-02-10 23:52:25,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,rsr i know i told you guys the target was i k...,104.0,False,0.000000,0.000000,0.193725
2,2021-02-10 23:45:03,#Bitcoin’s Rally Signals the Rise of Digital A...,s rally signals the rise of digital assets th...,14469.0,True,-0.200000,-1.664187,0.193725
3,2021-02-10 23:33:55,Our featured article for today! Please don't f...,our featured article for today please don t fo...,348.0,False,0.000000,0.000000,0.193725
4,2021-02-10 23:32:35,Who want to learn about Forex trading and join...,who want to learn about forex trading and join...,0.0,False,0.800000,0.000000,0.193725
...,...,...,...,...,...,...,...,...
51854,2022-03-14 19:40:26,"2- recession: the economical shift is coming, ...",recession the economical shift is coming the ...,149.0,False,0.237143,0.516044,0.380535
51855,2022-03-14 19:39:50,"@LadyofCrypto1 We're going to the moon, don't ...",we re going to the moon don t be late,202.0,False,-0.300000,-0.692249,0.380535
51856,2022-03-14 19:38:53,🛑Let's stop the war together!🛑\n📡Proceeds from...,let s stop the war together proceeds from the ...,8.0,False,0.000000,0.000000,0.380535
51857,2022-03-14 19:37:46,@patrickluberus @WhyyBitcoin @sathoarder @LOST...,the majority of users will be just watching t...,4315.0,False,0.200000,0.727016,0.380535


In [35]:
# data.to_csv('impact_score_avg_all.csv')